<a href="https://colab.research.google.com/github/Indresh0007/PySpark-Indresh/blob/main/Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## linux Basics

In [4]:
print("Hello")

Hello


In [6]:
!cat /etc/os-release


PRETTY_NAME="Ubuntu 22.04.4 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.4 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [7]:
!uname -a

Linux da354f09c9c4 6.1.123+ #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux


In [8]:
!whoami

root


In [11]:
!pwd

/content


In [12]:
pwd


'/content'

## PySpark Basics of DataFrame

In [13]:
!pip install pyspark

In [14]:
!pip show pyspark

Name: pyspark
Version: 3.5.1
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: py4j
Required-by: dataproc-spark-connect


In [15]:
from pyspark.sql import SparkSession

In [16]:
spark =SparkSession.builder.appName("Basics").getOrCreate()

In [19]:
# Create DataFrame
data=[("Hello", "World")]
columns=["Word1","Word2"]

df = spark.createDataFrame(data, columns)

In [20]:
df.show()


+-----+-----+
|Word1|Word2|
+-----+-----+
|Hello|World|
+-----+-----+



## Basics Transformation and Action

In [23]:
columns = ["Name","Department","Salary"]
data = [
    ("John", "Sales", 3000),
    ("Jane", "Finance", 4000),
    ("Mike", "Sales", 3500),
    ("Alice", "Finance", 3800),
    ("Bob", "IT", 4500)
]

df=spark.createDataFrame(data, columns)

In [24]:
df.show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
| John|     Sales|  3000|
| Jane|   Finance|  4000|
| Mike|     Sales|  3500|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [25]:
# Filter: Employee with salary > 3500
df_filtered=df.filter(df["Salary"]>3500)
df_filtered.show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
| Jane|   Finance|  4000|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [27]:
df_grouped=df.groupBy("Department").avg("Salary")
df_grouped.show()

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|     Sales|     3250.0|
|   Finance|     3900.0|
|        IT|     4500.0|
+----------+-----------+



In [28]:
#Add a new column: Salary with bonus (10%)
from pyspark.sql.functions import col
exp=col("Salary") * 1.1
df_with_bonus=df.withColumn("Salary_10%_bonus",exp)
df_with_bonus.show()

+-----+----------+------+------------------+
| Name|Department|Salary|  Salary_10%_bonus|
+-----+----------+------+------------------+
| John|     Sales|  3000|3300.0000000000005|
| Jane|   Finance|  4000|            4400.0|
| Mike|     Sales|  3500|3850.0000000000005|
|Alice|   Finance|  3800|            4180.0|
|  Bob|        IT|  4500|            4950.0|
+-----+----------+------+------------------+



In [34]:
from pyspark.sql.functions import col,upper,lower,concat_ws,when,length

df.show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
| John|     Sales|  3000|
| Jane|   Finance|  4000|
| Mike|     Sales|  3500|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [31]:
#Change Case transformation
df_upper = df.withColumn("Name_Upper", upper(col("Name")))
df_lower = df.withColumn("Name_Lower", lower(col("Name")))
df_upper.show()
df_lower.show()

+-----+----------+------+----------+
| Name|Department|Salary|Name_Upper|
+-----+----------+------+----------+
| John|     Sales|  3000|      JOHN|
| Jane|   Finance|  4000|      JANE|
| Mike|     Sales|  3500|      MIKE|
|Alice|   Finance|  3800|     ALICE|
|  Bob|        IT|  4500|       BOB|
+-----+----------+------+----------+

+-----+----------+------+----------+
| Name|Department|Salary|Name_Lower|
+-----+----------+------+----------+
| John|     Sales|  3000|      john|
| Jane|   Finance|  4000|      jane|
| Mike|     Sales|  3500|      mike|
|Alice|   Finance|  3800|     alice|
|  Bob|        IT|  4500|       bob|
+-----+----------+------+----------+



In [32]:
# Concatenate Columns
df_concat = df.withColumn("full_info", concat_ws("-", col("Name"), col("Department")))
df_concat.show()

+-----+----------+------+-------------+
| Name|Department|Salary|    full_info|
+-----+----------+------+-------------+
| John|     Sales|  3000|   John-Sales|
| Jane|   Finance|  4000| Jane-Finance|
| Mike|     Sales|  3500|   Mike-Sales|
|Alice|   Finance|  3800|Alice-Finance|
|  Bob|        IT|  4500|       Bob-IT|
+-----+----------+------+-------------+



In [35]:
# string length of names in new dataframe
df_length = df.withColumn("Name_Length", length(col("Name")))
df_length.show()

+-----+----------+------+-----------+
| Name|Department|Salary|Name_Length|
+-----+----------+------+-----------+
| John|     Sales|  3000|          4|
| Jane|   Finance|  4000|          4|
| Mike|     Sales|  3500|          4|
|Alice|   Finance|  3800|          5|
|  Bob|        IT|  4500|          3|
+-----+----------+------+-----------+



In [43]:
# conditional column (salary Category)
def_conditional = df.withColumn(
    "Salary_Category",
    when(col("Salary") >= 4000, "High")
    .when(col("Salary") >= 3500, "Medium")
    .otherwise("Low")

)
def_conditional.show()

+-----+----------+------+---------------+
| Name|Department|Salary|Salary_Category|
+-----+----------+------+---------------+
| John|     Sales|  3000|            Low|
| Jane|   Finance|  4000|           High|
| Mike|     Sales|  3500|         Medium|
|Alice|   Finance|  3800|         Medium|
|  Bob|        IT|  4500|           High|
+-----+----------+------+---------------+



In [3]:
# rename the column
def_renamed = df.withColumnRenamed("Salary", "Base_Salary")
def_renamed.show()

+-----+----------+-----------+
| Name|Department|Base_Salary|
+-----+----------+-----------+
| John|     Sales|       3000|
| Jane|   Finance|       4000|
| Mike|     Sales|       3500|
|Alice|   Finance|       3800|
|  Bob|        IT|       4500|
+-----+----------+-----------+



## GroupBy using PySpark

In [2]:
from pyspark.sql import SparkSession
spark =SparkSession.builder.appName("Basics").getOrCreate()
columns = ["Name","Department","Salary"]
data = [
    ("John", "Sales", 3000),
    ("Jane", "Finance", 4000),
    ("Mike", "Sales", 3500),
    ("Alice", "Finance", 3800),
    ("Bob", "IT", 4500)
]

df = spark.createDataFrame(data, columns)

df.show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
| John|     Sales|  3000|
| Jane|   Finance|  4000|
| Mike|     Sales|  3500|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [8]:
# Groupby Department and count employees
df.groupBy("Department").count().show()

+----------+-----+
|Department|count|
+----------+-----+
|     Sales|    2|
|   Finance|    2|
|        IT|    1|
+----------+-----+



In [6]:
# Groupby Department and calculate average salary
df.groupBy("Department").agg({"Salary": "avg"}).show()

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|     Sales|     3250.0|
|   Finance|     3900.0|
|        IT|     4500.0|
+----------+-----------+



In [9]:
#Group by Department and calculate multiple Aggregations
from pyspark.sql import functions as F

df.groupBy("Department") \
  .agg(F.sum("Salary").alias("TotalSalary"), \
       F.avg("Salary").alias("AverageSalary"), \
       F.min("Salary").alias("MinimumSalary"), \
       F.max("Salary").alias("MaximumSalary")) \
  .show()

+----------+-----------+-------------+-------------+-------------+
|Department|TotalSalary|AverageSalary|MinimumSalary|MaximumSalary|
+----------+-----------+-------------+-------------+-------------+
|     Sales|       6500|       3250.0|         3000|         3500|
|   Finance|       7800|       3900.0|         3800|         4000|
|        IT|       4500|       4500.0|         4500|         4500|
+----------+-----------+-------------+-------------+-------------+



## Joins using PySpark

In [15]:
# Create another DataFrame for department info
dept_data = [
    ("Sales", "Building A"),
    ("Finance", "Building B"),
    ("IT", "Building C")
]
dept_columns = ["Department", "Location"]

dept_df=spark.createDataFrame(dept_data, dept_columns)
dept_df.show()

+----------+----------+
|Department|  Location|
+----------+----------+
|     Sales|Building A|
|   Finance|Building B|
|        IT|Building C|
+----------+----------+



In [17]:
dept_df = spark.createDataFrame(dept_data, dept_columns)

#join employee data with department info
joined_df = df.join(dept_df, on="Department", how="inner")
joined_df.show()

+----------+-----+------+----------+
|Department| Name|Salary|  Location|
+----------+-----+------+----------+
|   Finance| Jane|  4000|Building B|
|   Finance|Alice|  3800|Building B|
|        IT|  Bob|  4500|Building C|
|     Sales| John|  3000|Building A|
|     Sales| Mike|  3500|Building A|
+----------+-----+------+----------+



In [18]:
#join employee data with department info
joined_df = df.join(dept_df, on="Department", how="left")
joined_df.show()

+----------+-----+------+----------+
|Department| Name|Salary|  Location|
+----------+-----+------+----------+
|     Sales| John|  3000|Building A|
|   Finance| Jane|  4000|Building B|
|     Sales| Mike|  3500|Building A|
|   Finance|Alice|  3800|Building B|
|        IT|  Bob|  4500|Building C|
+----------+-----+------+----------+



In [19]:
#join employee data with department info
joined_df = df.join(dept_df, on="Department", how="right")
joined_df.show()

+----------+-----+------+----------+
|Department| Name|Salary|  Location|
+----------+-----+------+----------+
|     Sales| Mike|  3500|Building A|
|     Sales| John|  3000|Building A|
|   Finance|Alice|  3800|Building B|
|   Finance| Jane|  4000|Building B|
|        IT|  Bob|  4500|Building C|
+----------+-----+------+----------+



In [20]:
# Employee DataFrame
emp_data = [
    (1, "John", "Sales", 3000),
    (2, "Jane", "Finance", 4000),
    (3, "Mike", "Sales", 3500),
    (4, "Alice", "HR", 3800),
    (5, "Bob", "IT", 4500),
    (6, "Sam", "Support", 3200)
]
emp_cols = ["EmpID", "Name", "Department", "Salary"]
emp_df = spark.createDataFrame(emp_data, emp_cols)

# Department DataFrame
dept_data = [
    ("Sales", "Building A"),
    ("Finance", "Building B"),
    ("IT", "Building C"),
    ("Admin", "Building D")
]
dept_cols = ["Department", "Location"]
dept_df = spark.createDataFrame(dept_data, dept_cols)

# Display both
emp_df.show()
dept_df.show()

+-----+-----+----------+------+
|EmpID| Name|Department|Salary|
+-----+-----+----------+------+
|    1| John|     Sales|  3000|
|    2| Jane|   Finance|  4000|
|    3| Mike|     Sales|  3500|
|    4|Alice|        HR|  3800|
|    5|  Bob|        IT|  4500|
|    6|  Sam|   Support|  3200|
+-----+-----+----------+------+

+----------+----------+
|Department|  Location|
+----------+----------+
|     Sales|Building A|
|   Finance|Building B|
|        IT|Building C|
|     Admin|Building D|
+----------+----------+



In [21]:
emp_df.join(dept_df, on="Department", how="inner").show()

+----------+-----+----+------+----------+
|Department|EmpID|Name|Salary|  Location|
+----------+-----+----+------+----------+
|   Finance|    2|Jane|  4000|Building B|
|        IT|    5| Bob|  4500|Building C|
|     Sales|    1|John|  3000|Building A|
|     Sales|    3|Mike|  3500|Building A|
+----------+-----+----+------+----------+



In [23]:
emp_df.join(dept_df, on="Department", how="left").show()

+----------+-----+-----+------+----------+
|Department|EmpID| Name|Salary|  Location|
+----------+-----+-----+------+----------+
|     Sales|    1| John|  3000|Building A|
|     Sales|    3| Mike|  3500|Building A|
|   Finance|    2| Jane|  4000|Building B|
|        HR|    4|Alice|  3800|      NULL|
|        IT|    5|  Bob|  4500|Building C|
|   Support|    6|  Sam|  3200|      NULL|
+----------+-----+-----+------+----------+



In [24]:
emp_df.join(dept_df, on="Department", how="right").show()

+----------+-----+----+------+----------+
|Department|EmpID|Name|Salary|  Location|
+----------+-----+----+------+----------+
|     Sales|    3|Mike|  3500|Building A|
|     Sales|    1|John|  3000|Building A|
|   Finance|    2|Jane|  4000|Building B|
|     Admin| NULL|NULL|  NULL|Building D|
|        IT|    5| Bob|  4500|Building C|
+----------+-----+----+------+----------+



In [25]:
emp_df.join(dept_df, on="Department", how="full").show()

+----------+-----+-----+------+----------+
|Department|EmpID| Name|Salary|  Location|
+----------+-----+-----+------+----------+
|     Admin| NULL| NULL|  NULL|Building D|
|   Finance|    2| Jane|  4000|Building B|
|        HR|    4|Alice|  3800|      NULL|
|        IT|    5|  Bob|  4500|Building C|
|     Sales|    1| John|  3000|Building A|
|     Sales|    3| Mike|  3500|Building A|
|   Support|    6|  Sam|  3200|      NULL|
+----------+-----+-----+------+----------+



## Assessment

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, avg, count, max as _max, min as _min, rank
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("SalesAnalysis").getOrCreate()

# Sales Dataset
sales_data = [
    (1001, 101, 501, "2025-07-10 08:23:00", 3, 25.5),
    (1002, 102, 502, "2025-07-11 09:45:00", 2, 15.0),
    (1003, 103, 503, "2025-07-12 10:15:00", 1, 30.0),
    (1004, 101, 504, "2025-07-13 12:20:00", 5, 25.5),
    (1005, 105, 505, "2025-07-14 14:35:00", 10, 45.0),
    (1006, 102, 506, "2025-07-15 16:00:00", 4, 15.0),
]
sales_cols = ["sale_id", "product_id", "customer_id", "sale_date", "quantity", "price"]
sales_df = spark.createDataFrame(sales_data, sales_cols)

# Product Dataset
product_data = [
    (101, "Widget A", "Gadgets"),
    (102, "Widget B", "Gadgets"),
    (103, "Widget C", "Electronics"),
    (104, "Widget D", "Electronics"),
    (105, "Widget E", "Home & Living"),
]
product_cols = ["product_id", "product_name", "category"]
product_df = spark.createDataFrame(product_data, product_cols)

# Customer Dataset
customer_data = [
    (501, "Alice", "alice@example.com", "2025-05-20 10:10:00"),
    (502, "Bob", "bob@example.com", "2025-06-15 14:00:00"),
    (503, "Charlie", "charlie@example.com", "2025-04-05 09:50:00"),
    (504, "David", "david@example.com", "2025-07-01 12:25:00"),
    (505, "Emma", "emma@example.com", "2025-07-10 15:30:00"),
    (506, "Frank", "frank@example.com", "2025-03-23 17:00:00"),
]
customer_cols = ["customer_id", "customer_name", "email", "join_date"]
customer_df = spark.createDataFrame(customer_data, customer_cols)


Aggregations

In [27]:
sales_df.withColumn("revenue", col("quantity") * col("price")) \
    .groupBy("product_id") \
    .agg(_sum("revenue").alias("total_revenue")) \
    .show()


+----------+-------------+
|product_id|total_revenue|
+----------+-------------+
|       103|         30.0|
|       101|        204.0|
|       102|         90.0|
|       105|        450.0|
+----------+-------------+



In [28]:
sales_df.groupBy("customer_id").agg(_sum("quantity").alias("total_qty")).show()


+-----------+---------+
|customer_id|total_qty|
+-----------+---------+
|        502|        2|
|        501|        3|
|        503|        1|
|        504|        5|
|        506|        4|
|        505|       10|
+-----------+---------+



In [29]:
sales_df.withColumn("revenue", col("quantity") * col("price")) \
    .groupBy("customer_id") \
    .agg(avg("revenue").alias("avg_revenue")) \
    .show()


+-----------+-----------+
|customer_id|avg_revenue|
+-----------+-----------+
|        502|       30.0|
|        501|       76.5|
|        503|       30.0|
|        504|      127.5|
|        506|       60.0|
|        505|      450.0|
+-----------+-----------+



In [30]:
from pyspark.sql.functions import to_date, date_format

sales_df.withColumn("month", date_format("sale_date", "yyyy-MM")) \
    .withColumn("revenue", col("quantity") * col("price")) \
    .groupBy("month").agg(_sum("revenue").alias("monthly_sales")) \
    .show()


+-------+-------------+
|  month|monthly_sales|
+-------+-------------+
|2025-07|        774.0|
+-------+-------------+



In [31]:
sales_df.join(product_df, "product_id") \
    .groupBy("category").agg(count("*").alias("total_sales")) \
    .show()


+-------------+-----------+
|     category|total_sales|
+-------------+-----------+
|  Electronics|          1|
|      Gadgets|          4|
|Home & Living|          1|
+-------------+-----------+



In [32]:
sales_df.withColumn("revenue", col("quantity") * col("price")) \
    .join(product_df, "product_id") \
    .groupBy("product_id", "product_name") \
    .agg(_sum("revenue").alias("total_revenue")) \
    .orderBy(col("total_revenue").desc()) \
    .limit(3) \
    .show()


+----------+------------+-------------+
|product_id|product_name|total_revenue|
+----------+------------+-------------+
|       105|    Widget E|        450.0|
|       101|    Widget A|        204.0|
|       102|    Widget B|         90.0|
+----------+------------+-------------+



Joins

In [33]:
sales_df.join(product_df, "product_id").show()


+----------+-------+-----------+-------------------+--------+-----+------------+-------------+
|product_id|sale_id|customer_id|          sale_date|quantity|price|product_name|     category|
+----------+-------+-----------+-------------------+--------+-----+------------+-------------+
|       101|   1001|        501|2025-07-10 08:23:00|       3| 25.5|    Widget A|      Gadgets|
|       101|   1004|        504|2025-07-13 12:20:00|       5| 25.5|    Widget A|      Gadgets|
|       102|   1002|        502|2025-07-11 09:45:00|       2| 15.0|    Widget B|      Gadgets|
|       102|   1006|        506|2025-07-15 16:00:00|       4| 15.0|    Widget B|      Gadgets|
|       103|   1003|        503|2025-07-12 10:15:00|       1| 30.0|    Widget C|  Electronics|
|       105|   1005|        505|2025-07-14 14:35:00|      10| 45.0|    Widget E|Home & Living|
+----------+-------+-----------+-------------------+--------+-----+------------+-------------+



In [34]:
sales_df.join(customer_df, "customer_id").show()


+-----------+-------+----------+-------------------+--------+-----+-------------+-------------------+-------------------+
|customer_id|sale_id|product_id|          sale_date|quantity|price|customer_name|              email|          join_date|
+-----------+-------+----------+-------------------+--------+-----+-------------+-------------------+-------------------+
|        501|   1001|       101|2025-07-10 08:23:00|       3| 25.5|        Alice|  alice@example.com|2025-05-20 10:10:00|
|        502|   1002|       102|2025-07-11 09:45:00|       2| 15.0|          Bob|    bob@example.com|2025-06-15 14:00:00|
|        503|   1003|       103|2025-07-12 10:15:00|       1| 30.0|      Charlie|charlie@example.com|2025-04-05 09:50:00|
|        504|   1004|       101|2025-07-13 12:20:00|       5| 25.5|        David|  david@example.com|2025-07-01 12:25:00|
|        505|   1005|       105|2025-07-14 14:35:00|      10| 45.0|         Emma|   emma@example.com|2025-07-10 15:30:00|
|        506|   1006|   

In [35]:
sales_df.join(product_df, "product_id") \
    .filter(col("category") == "Gadgets") \
    .show()


+----------+-------+-----------+-------------------+--------+-----+------------+--------+
|product_id|sale_id|customer_id|          sale_date|quantity|price|product_name|category|
+----------+-------+-----------+-------------------+--------+-----+------------+--------+
|       101|   1001|        501|2025-07-10 08:23:00|       3| 25.5|    Widget A| Gadgets|
|       101|   1004|        504|2025-07-13 12:20:00|       5| 25.5|    Widget A| Gadgets|
|       102|   1002|        502|2025-07-11 09:45:00|       2| 15.0|    Widget B| Gadgets|
|       102|   1006|        506|2025-07-15 16:00:00|       4| 15.0|    Widget B| Gadgets|
+----------+-------+-----------+-------------------+--------+-----+------------+--------+



In [36]:
sales_df.join(product_df, "product_id", "left").show()


+----------+-------+-----------+-------------------+--------+-----+------------+-------------+
|product_id|sale_id|customer_id|          sale_date|quantity|price|product_name|     category|
+----------+-------+-----------+-------------------+--------+-----+------------+-------------+
|       103|   1003|        503|2025-07-12 10:15:00|       1| 30.0|    Widget C|  Electronics|
|       101|   1001|        501|2025-07-10 08:23:00|       3| 25.5|    Widget A|      Gadgets|
|       102|   1002|        502|2025-07-11 09:45:00|       2| 15.0|    Widget B|      Gadgets|
|       105|   1005|        505|2025-07-14 14:35:00|      10| 45.0|    Widget E|Home & Living|
|       101|   1004|        504|2025-07-13 12:20:00|       5| 25.5|    Widget A|      Gadgets|
|       102|   1006|        506|2025-07-15 16:00:00|       4| 15.0|    Widget B|      Gadgets|
+----------+-------+-----------+-------------------+--------+-----+------------+-------------+



In [39]:
sales_df.join(product_df, "product_id", "outer").show()


+----------+-------+-----------+-------------------+--------+-----+------------+-------------+
|product_id|sale_id|customer_id|          sale_date|quantity|price|product_name|     category|
+----------+-------+-----------+-------------------+--------+-----+------------+-------------+
|       101|   1001|        501|2025-07-10 08:23:00|       3| 25.5|    Widget A|      Gadgets|
|       101|   1004|        504|2025-07-13 12:20:00|       5| 25.5|    Widget A|      Gadgets|
|       102|   1002|        502|2025-07-11 09:45:00|       2| 15.0|    Widget B|      Gadgets|
|       102|   1006|        506|2025-07-15 16:00:00|       4| 15.0|    Widget B|      Gadgets|
|       103|   1003|        503|2025-07-12 10:15:00|       1| 30.0|    Widget C|  Electronics|
|       104|   NULL|       NULL|               NULL|    NULL| NULL|    Widget D|  Electronics|
|       105|   1005|        505|2025-07-14 14:35:00|      10| 45.0|    Widget E|Home & Living|
+----------+-------+-----------+------------------

In [40]:
sales_df.join(customer_df, "customer_id") \
    .join(product_df, "product_id") \
    .select("sale_id", "customer_name", "product_name", "category", "quantity", "price") \
    .show()


+-------+-------------+------------+-------------+--------+-----+
|sale_id|customer_name|product_name|     category|quantity|price|
+-------+-------------+------------+-------------+--------+-----+
|   1004|        David|    Widget A|      Gadgets|       5| 25.5|
|   1001|        Alice|    Widget A|      Gadgets|       3| 25.5|
|   1006|        Frank|    Widget B|      Gadgets|       4| 15.0|
|   1002|          Bob|    Widget B|      Gadgets|       2| 15.0|
|   1003|      Charlie|    Widget C|  Electronics|       1| 30.0|
|   1005|         Emma|    Widget E|Home & Living|      10| 45.0|
+-------+-------------+------------+-------------+--------+-----+



In [42]:
s1 = sales_df.alias("s1")
s2 = sales_df.alias("s2")

s1.join(
    s2,
    (col("s1.product_id") == col("s2.product_id")) & (col("s1.sale_date") < col("s2.sale_date"))
).select(
    col("s1.sale_id").alias("sale_id_1"),
    col("s1.product_id"),
    col("s1.sale_date").alias("first_sale_date"),
    col("s2.sale_date").alias("later_sale_date")
).show()


+---------+----------+-------------------+-------------------+
|sale_id_1|product_id|    first_sale_date|    later_sale_date|
+---------+----------+-------------------+-------------------+
|     1001|       101|2025-07-10 08:23:00|2025-07-13 12:20:00|
|     1002|       102|2025-07-11 09:45:00|2025-07-15 16:00:00|
+---------+----------+-------------------+-------------------+



Additional Transformations

In [43]:
sales_df.filter((col("sale_date") >= "2025-07-10") & (col("sale_date") <= "2025-07-15")).show()


+-------+----------+-----------+-------------------+--------+-----+
|sale_id|product_id|customer_id|          sale_date|quantity|price|
+-------+----------+-----------+-------------------+--------+-----+
|   1001|       101|        501|2025-07-10 08:23:00|       3| 25.5|
|   1002|       102|        502|2025-07-11 09:45:00|       2| 15.0|
|   1003|       103|        503|2025-07-12 10:15:00|       1| 30.0|
|   1004|       101|        504|2025-07-13 12:20:00|       5| 25.5|
|   1005|       105|        505|2025-07-14 14:35:00|      10| 45.0|
+-------+----------+-----------+-------------------+--------+-----+



In [44]:
spend_df = sales_df.withColumn("spend", col("quantity") * col("price")) \
    .groupBy("customer_id").agg(_sum("spend").alias("total_spend"))

spend_df.orderBy(col("total_spend").desc()).limit(5).show()


+-----------+-----------+
|customer_id|total_spend|
+-----------+-----------+
|        505|      450.0|
|        504|      127.5|
|        501|       76.5|
|        506|       60.0|
|        502|       30.0|
+-----------+-----------+



In [45]:
from pyspark.sql.functions import when

categorized = spend_df.withColumn("category",
    when(col("total_spend") < 100, "Low")
    .when(col("total_spend") < 200, "Medium")
    .otherwise("High"))

categorized.show()


+-----------+-----------+--------+
|customer_id|total_spend|category|
+-----------+-----------+--------+
|        502|       30.0|     Low|
|        501|       76.5|     Low|
|        503|       30.0|     Low|
|        504|      127.5|  Medium|
|        506|       60.0|     Low|
|        505|      450.0|    High|
+-----------+-----------+--------+



In [46]:
from pyspark.sql.functions import min as _min, max as _max

sales_df.groupBy("customer_id") \
    .agg(_min("sale_date").alias("first_purchase"), _max("sale_date").alias("last_purchase")) \
    .show()


+-----------+-------------------+-------------------+
|customer_id|     first_purchase|      last_purchase|
+-----------+-------------------+-------------------+
|        501|2025-07-10 08:23:00|2025-07-10 08:23:00|
|        502|2025-07-11 09:45:00|2025-07-11 09:45:00|
|        503|2025-07-12 10:15:00|2025-07-12 10:15:00|
|        504|2025-07-13 12:20:00|2025-07-13 12:20:00|
|        505|2025-07-14 14:35:00|2025-07-14 14:35:00|
|        506|2025-07-15 16:00:00|2025-07-15 16:00:00|
+-----------+-------------------+-------------------+



In [48]:
from pyspark.sql.functions import current_date, datediff, to_date, lit, max as _max, col

latest_date = sales_df.agg(_max("sale_date")).collect()[0][0]

sales_df.withColumn("sale_dt", to_date("sale_date")) \
    .groupBy("customer_id") \
    .agg(_max("sale_dt").alias("last_purchase")) \
    .filter(datediff(to_date(lit(latest_date)), col("last_purchase")) > 30) \
    .show()


+-----------+-------------+
|customer_id|last_purchase|
+-----------+-------------+
+-----------+-------------+



In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, avg, count, max as _max, min as _min, rank, to_date, date_format, lit, datediff, when
from pyspark.sql.window import Window

# Create Spark Session
spark = SparkSession.builder.appName("SalesAnalysis").getOrCreate()


In [50]:
sales_df = spark.read.option("header", "true").option("inferSchema", "true").csv("/content/sales_data.csv")
product_df = spark.read.option("header", "true").option("inferSchema", "true").csv("/content/product_data.csv")
customer_df = spark.read.option("header", "true").option("inferSchema", "true").csv("/content/customer_data.csv")

sales_df.show()
product_df.show()
customer_df.show()


+-------+----------+-----------+-------------------+--------+-----+
|sale_id|product_id|customer_id|          sale_date|quantity|price|
+-------+----------+-----------+-------------------+--------+-----+
|   1001|       101|        501|2025-07-10 08:23:00|       3| 25.5|
|   1002|       102|        502|2025-07-11 09:45:00|       2| 15.0|
|   1003|       103|        503|2025-07-12 10:15:00|       1| 30.0|
|   1004|       101|        504|2025-07-13 12:20:00|       5| 25.5|
|   1005|       105|        505|2025-07-14 14:35:00|      10| 45.0|
|   1006|       102|        506|2025-07-15 16:00:00|       4| 15.0|
|   1007|       106|        507|2025-07-16 17:10:00|       3| 40.0|
|   1008|       107|        508|2025-07-17 18:25:00|       2| 60.0|
|   1009|       108|        509|2025-07-18 19:30:00|       7| 25.0|
|   1010|       109|        510|2025-07-19 20:45:00|       6| 50.0|
|   1011|       110|        511|2025-07-20 21:55:00|       4| 12.5|
|   1012|       111|        512|2025-07-21 22:30

In [51]:
sales_df.withColumn("revenue", col("quantity") * col("price")) \
    .groupBy("product_id") \
    .agg(_sum("revenue").alias("total_revenue")) \
    .show()


+----------+-------------+
|product_id|total_revenue|
+----------+-------------+
|       148|         58.0|
|       137|        115.0|
|       133|        115.5|
|       108|        235.0|
|       101|        280.5|
|       115|        484.0|
|       126|        240.0|
|       103|        240.0|
|       128|         90.0|
|       122|        100.0|
|       111|        292.5|
|       140|         80.0|
|       132|        132.0|
|       146|        180.0|
|       142|        190.0|
|       139|         55.5|
|       120|        211.5|
|       117|        114.0|
|       112|        231.0|
|       127|         70.0|
+----------+-------------+
only showing top 20 rows

